In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
この実習で使うサンプルファイルをダウンロードします。
"""
!wget https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_bulk.txt -O mutmap_bulk.txt

# 大規模データ解析入門 (2)
pandasを使ったデータ解析とグラフ作成

## Contents

### Introduction
- [前回の復習](#0.1)
- [今回のサンプルデータ](#0.2)

--- 

### Practice
1. [pandasデータフレームとしてファイルを読み込む](#2.1)
1. [任意のデータにアクセスする](#2.2)
1. [SNP-indexを計算する](#2.3)
1. [条件に合うデータを抽出する](#2.4)
1. [ファイルに書き出す](#2.5)
1. [SNP-indexのグラフを描く](#2.6)
1. [Sliding Window解析](#2.7)

## Introduction

### 前回の復習と今回の実習<a name="0.1"></a>

前回、テキストデータ処理の基本を勉強しました。   
=> [前回のページ](../06_large_data_analysis/01_large_data_analysis_ja.ipynb#) 

`for`文を使い、「上の行から順番に、一行ずつデータを読み込み、行内を分割し、データ処理をおこなう」ことで全データを処理しました。  
（さらに、課題では`if`文を使って、任意のデータのみを得る、といったこともおこないました。）  

そのような方法は、テキストデータだけに限らず、あらゆるデータに対して適用可能な基本的な処理方法ですが、処理速度が遅い、コードが長くなりがち、といった欠点があります。

テーブル状に整ったデータであれば、[pandas](https://pandas.pydata.org/)ライブラリを利用することで、より高速かつ簡単にデータ処理が可能になります。  

今回の実習では、
1. pandasを使ってデータ処理をしてみましょう。  
1. さらに、[Matplotlib](https://matplotlib.org/)ライブラリを利用して、データをグラフにしてみましょう。

### 今回のサンプルデータ<a name="0.2"></a>

前回と同じく、MutMap (Abe et al. 2012) のデータを使います。  
=> [前回のページ / サンプルデータの説明](../06_large_data_analysis/01_large_data_analysis_ja.ipynb#0.3) 

データは次のようにテーブル状になった「タブ区切りテキストファイル」です。  
_列と列の間が、「タブ」と呼ばれる特殊文字（Macの場合 `\t`; Windowsの場合 `¥t`）で区切られたテキストファイルです。_

右の2つの列から「SNP-index」と呼ばれる値を算出することが可能です。

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/07/tab-sep_text_ja.jpg?raw=true" alt="matplotlib_graph"></div>

実際のファイルの中身も確認してください => [mutmap_bulk.txt](mutmap_bulk.txt)

---
## Practice

#### 実習内容
1. [pandasデータフレームとしてファイルを読み込む](#2.1)
1. [任意のデータにアクセスする](#2.2)
1. [SNP-indexを計算する](#2.3)
1. [条件に合うデータを抽出する](#2.4)
1. [ファイルに書き出す](#2.5)
1. [SNP-indexのグラフを描く](#2.6)
1. [Sliding Window解析](#2.7)

プログラムの大部分はすでに書かれていますが、いくつかコードを書き足す部分があります。  
書き足す部分はこのように書かれています。
```python
# !!! 以下にプログラムを追記する !!!
```

### 1. Pandasデータフレームとしてファイルを読み込む<a name="1.1"></a>

pandasの`read_csv`関数でテーブル状のテキストファイルを読み込みます。

=== 基本構文 ===
```python
import pandas
df = pandas.read_csv("<ファイル名>", sep="<区切り文字>", header=<列名の行番号>)
```
または、
```python
import pandas as pd
df = pd.read_csv("<ファイル名>", sep="<区切り文字>", header=<列名の行番号>)
```

=== 説明 ===

#### `import pandas`について
- pandasライブラリをプログラムで使えるように準備しています。
- `import pandas`とした場合、以降のプログラムで、pandasが持つ関数を `pandas.関数名`の形で使用できます。
- `import pandas as pd`とした場合、以降のプログラムで、pandasが持つ関数を `pd.関数名`の形で使用できます。  
  _（pandasは省略名として`pd`がよく使われます。）_
        
        
#### `df = pandas.read_csv("<ファイル名>", sep="<区切り文字>", header=<列名の行番号>)`について

- 第一引数にファイル名
- `sep=`では、読み込みファイルで使われている区切り文字を指定します。
    * カンマ区切り: `,`
    * タブ区切り: `\t`（Mac）、`¥t`（Windows）
- `header=`では、テーブルの列名にあたる行番号を指定します。
    * 最初の行が列名の場合`0`を指定します。
    * 列名の行がない場合`-1`を指定します。
- 読み込まれたテーブルは「データフレーム」と呼ばれる形式で`df`に保持されます。（`df`は変数名なので、他の名前でも良い）

In [ ]:
#--- pandasを読み込む  ---
import pandas as pd

#--- 読み込みファイル名 ---
dataset = 'mutmap_bulk.txt'       

#--- ファイルを読み込む ---
# header=-1（列名行なし）を指定した場合、列名として通し番号が付けられます。
# names=[リスト]とすることで、通し番号の代わりに列名を指定することが可能です。
df = pd.read_csv(dataset, sep="\t", header=-1, names=['chr', 'pos', 'ref_nucl', 'alt_nucl', 'ref_N', 'alt_N'])

#--- 表示 ---
df

### 補足説明1
#### データフレームの見方
- エクセルのような表（テーブル）になっています。
- 一番上の行は、各列の名前です。
- 一番左の列は、各行の名前（「__インデックス__」という）です。

### 補足説明2
#### `print()`を使っていないのに、なぜ表示されるのか？
Jupyter Notebookの機能です。  
Jupyter Notebookでは、セル内のプログラムの最後に変数名のみを書くことで、その変数の中身が確認可能です。  
ただし、表示できる変数は最後の一つだけです。  

In [ ]:
a = 1
b = 2
c = 3

a
b
c

### 2. 任意のデータにアクセスする<a name="1.2"></a>
データフレームの任意のデータにアクセスしてみます。  

適宜、各コードの`#`を外して、実行結果を確認しましょう。


__＊注意＊__  
Jupyter Notebookでは、上のセルが作成された変数等が下のセルに引き継がれます。  
以下のセルのプログラムは、「1. Pandasデータフレームとしてファイルを読み込む」のセルが一度実行されたことを前提に書かれています。  

In [ ]:
###### 全て表示（途中のデータは省略されます） ######
df

###### 1列を抽出 ######
#df['ref_nucl']
#df.loc[:, 'ref_nucl']
#df.iloc[:, 2]


###### 複数列を抽出 ######
#df.loc[:, ['ref_nucl','alt_nucl']]
#df.iloc[:, 2:4]


###### 行の抽出 ######
#df.loc[10,:]
#df.iloc[10,:]


###### 複数行を抽出 ######
#df.loc[10:15, :]
#df.iloc[10:15, :]


###### 1セル抽出 ######
#df.loc[10, 'ref_nucl']
#df.iloc[10, 2]


###### 複数列と複数行を抽出 ######
#df.loc[10:15, ['ref_nucl', 'alt_nucl']]
#df.iloc[10:15, 2:4]


### 3. SNP-indexを計算する<a name="1.3"></a>

前回の実習では、`for`や`split()`を使って、一行ごとに`ref_N`や`alt_N`にあたるデータにアクセスして、SNP-indexを計算しました。  
今回は、`for`を使わずに、列同士の計算をおこなってみましょう。  

#### 方法  
計算に使う列を取り出し、計算式に直接入れる。

#### SNP-index計算式
SNP-index = alt_N / (ref_N + alt_N)


In [ ]:
###### SNP-indexの計算 ######

# !!! 以下にプログラムを追記する !!!
#df['snp_index'] = 

#--- 表示 ---
df

完全版のプログラムは[こちら](./02_large_data_analysis_ja_complete_version.ipynb#1.3)

_サンプルデータは行数が少ないために実感しずらいですが、今回の列同士の計算は`for`を使った方法よりも高速に実行されます。  また、コード自体もシンプルで、理解しやすいかと思います。_

### 4. 条件に合うデータを抽出する<a name="1.4"></a>

=== 基本構文 ===

```python
df[ (条件式) ]
```

In [ ]:
###### 条件を指定して抽出 ######

#--- 条件をひとつ指定 ---
# df['ref_nucl']=='A'    # これだけでは望んだ結果が得られない

# df[ df['ref_nucl']=='A' ] 

#--- 複数の条件を指定 ---
# df[ (df['ref_nucl']=='A' ) & (df['alt_nucl']=='G' ) ]    # AND

# df[ (df['ref_nucl']=='A' ) | (df['alt_nucl']=='G' ) ]    # OR


#--- SNP-indexが0.9以上のデータを抽出 ---
# !!! 以下にプログラムを追記する !!!



完全版のプログラムは[こちら](./02_large_data_analysis_ja_complete_version.ipynb#1.4)

### 補足説明3
#### `df['ref_nucl']=='A' `と`df[ df['ref_nucl']=='A' ] `について

`df['ref_nucl']=='A' `で、条件に合うデータを`True`、合わないデータを`False`としたリスト状のデータが得られます。

さらに、`df[ (True/Falseリスト) ]`の形にすることで、`True`のデータのみを表示させています。

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/07/pandas_filtering_ja.jpg?raw=true" alt="pandas_filtering"></div>

### 5. ファイルに書き出す<a name="1.5"></a>

ファイルへの出力には、pandasの`to_csv`関数を利用します。

=== 基本構文 ===

```python
データフレーム.to_csv("<ファイル名>", sep="<区切り文字>", header=<True/False>, index=<True/False>)
```

=== 説明 ===
- 第一引数にファイル名
- sep=では、列間の区切り文字を指定します。
    * カンマ区切り: `,`
    * タブ区切り: `\t`（Mac）、`¥t`（Windows）
- `header=True`にすれば、列名がファイルに書き込まれます。
- `index=True`にすれば、行名（インデックス）がファイルに書き込まれます。

In [ ]:
###### ファイルの書き出し ######
#outdata = 'mutmap_snpindex_using_pd.txt'        # 書き出しファイル名
#df.to_csv(outdata, sep='\t', header=True, index=False)

出力されたファイルを確認してみましょう => [ファイル一覧へ](./)

### 6. SNP-indexのグラフを描く<a name="1.6"></a>

ここでは、Pythonのグラフ描画で有名なライブラリ [Matplotlib](https://matplotlib.org/) を使ってグラフを描いてみましょう。

まず、グラフ描画の基本についてです。

matplotlibライブラリのグラフは、「レイヤー」と呼ばれる透明なシートを複数重ねることで、ひとつのグラフが作成されます。  
それぞれのレイヤーには、グラフフィールド（下地）、グラフ（例えば折れ線グラフ）、軸ラベル（文字）等が描かれています。

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/07/07_drawing_graph.png?raw=true" alt="matplotlib_graph"></div>

今回は、x軸に染色体上の位置、y軸にSNP-index（「3. SNP-indexを計算する」で求めた）をとったグラフ（散布図）を作成してみましょう。  
加えて、「SNP-index >= 0.9」の点を赤で表示させてみましょう。

In [ ]:
###### SNP-indexのグラフ作成 ######
"""
この1行はmatplotlibを使ったグラフを、Jupyter Notebook上で表示させるために必要です。
Pythonプログラムではありません。Jupyter Notebookの「マジックコマンド」と言われるものです。
"""
%matplotlib inline


"""
ここからPythonプログラム
"""
#--- ライブラリを読み込む ---
import matplotlib.pyplot as plt

#--- x軸データとy軸データ ---
df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])
x = df['pos']
y = df['snp_index']

#--- グラフ描画 ---
fig = plt.figure(figsize=[16,9])    # グラフフィールドの設定
plt.scatter(x, y, color='gray')      # 散布図
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)           # y軸ラベル


#--- グラフにsnp_index>=0.9のデータを重ね描き ---
# !!! 以下にプログラムを追記する !!!


完全版のプログラムは[こちら](./02_large_data_analysis_ja_complete_version.ipynb#1.6)

### 7. Sliding Window解析<a name="1.7"></a>
MutMap解析では、SNP-indexの区間平均をゲノムに全体にわたって調べる解析（Sliding Window解析）というのをおこない、形質にかかわるゲノム領域を特定します。 

_形質にかかわる領域付近では、直接の原因となる塩基座位だけでなく、周囲の塩基座位のSNP-indexも1に近づきます。_


<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/07/sliding_window_ja.jpg?raw=true" alt="matplotlib_graph"></div>


ここでは上の図のように、Chromosome 10上のSNP-indexの推移を調べてみましょう。  
あらかじめプログラムを書いています。  

#### 解析方法
SNP-indexのSliding Window解析は次のような手順で解析します。
1. 区間サイズとステップサイズを決める。
1. 各区間の代表値（ここでは区間の中央値）用のリストと、区間内の平均SNP-index用のリストを用意する。  
1. `while`文を使って全ての区間を調べていく。  
    1. 区間内に含まれるデータを抽出する。
    1. 抽出したデータからSNP-indexの平均値を算出する。
    1. 区間の中央値と算出した平均SNP-indexを、それぞれのリストに追加する。
    1. ゲノムの全長を全て調べたら`while`から出る
1. x軸に中央値、y軸に平均SNP-indexの値をとったグラフ（折れ線グラフなど）を作成する。

In [ ]:
###### Sliding Window解析 ######
#---  NumPyライブラリの読み込み ---
import numpy as np

#--- 区間サイズとステップサイズ ---
CHROM_SIZE = 23207287       # Chromosome 10　の全長 (bp)
WIN_SIZE       = 1 * 1000 * 1000     #  1 Mb = 1000 kb = 1,000,000 bp
STEP_SIZE     = 0.2 * 1000* 1000     #  0.2 Mb = 200 kb = 200,000 bp

#--- 区間データ用のリストを準備 ---
win_position  = []  # 区間中央値用リスト
win_snpindex = []  # 平均SNP-index用リスト

#--- 全ての区間を調べる ---
"""
/// 調べる区間 ///
start, end
0, 0+1000 (kb)
200, 200+1000
400, 400+1000
  .
  .
  .

/// WIN_SIZEとSTEP_SIZEを使って表現すると... ///
繰り返し数:　n = 0, 1, 2, ...

start = STEP_SIZE * n  
end = start + WIN_SIZE


end > CHROM_SIZEになったとき、繰り返しを終える。

このように、繰り返し数を指定しない場合は、for文ではなく、while文を使用します。
"""

n = 0 # 繰り返し数
while True:
    
    #--- 区間のstartとend position ---
    start = STEP_SIZE * n 
    end   = start + WIN_SIZE
    
    #--- 区間の中央値をリストに追加する ---
    p = (start + end) / 2
    win_position.append(p)
    
    #--- 区間内データを抽出 ---
    sub = df[(df['pos'] >= start) & (df['pos'] < end)]
    
    #--- SNP-indexの平均値を算出 ---
    i = sub['snp_index'].mean()
    win_snpindex.append(i)
        
    #--- 繰り返し数を+1 ---
    n += 1
    
    #--- 全ての区間を調べた時、whileから出る ---
    if end > CHROM_SIZE:
        break

#--- グラフ作成  ---
fig = plt.figure(figsize=[16,9])
plt.scatter(x, y, color='gray')      # 全データ
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)        # y軸ラベル

#--- グラフにsnp_index>=0.9のデータを重ね描き ---
df_ext = df[ df['snp_index'] >= 0.9 ]
x1 = df_ext['pos']
y1 = df_ext['snp_index']
plt.scatter(x1, y1, color='red')

#--- 平均SNP-indexの折れ線グラフ（重ね描き） ---
plt.plot(win_position, win_snpindex, color='blue')      

平均SNP-indexが0.9以上の区間を原因遺伝子の候補領域として取り出しましょう。

In [ ]:
### 平均SNP-indexが0.9以上の区間を取り出す ###

#--- Sliding windowのデータをpandasデータフレームにする ---
sw = pd.DataFrame({ 'pos': win_position, 'snp_index': win_snpindex})

#--- 区間のstartとendの位置 ---
sw['start'] = sw['pos'] - WIN_SIZE / 2 
sw['end'] = sw['pos'] + WIN_SIZE / 2

#--- 平均SNP-indexが0.9以上の区間 ---
sw[sw['snp_index'] >= 0.9] 

上の例では、平均SNP-indexの値が0.9以上のデータを取り出しました。  
なぜ「0.9」でしょうか？ 「0.8」や「0.89」、「0.91」ではダメなのでしょうか？  
この「0.9」は "なんとなく選んだ" 数値（閾値）です。つまり、あまり根拠がなく主観的です。  

もう少し根拠のある閾値を設定することはできないのでしょうか？  

シミュレーションをおこなうことで、ある値のSNP-indexが得られる確率を調べることができます。  
シミュレーションにより得られた確率に基づいて閾値を設定することで、より客観的にデータ（得られたSNP-index）を評価することが可能となります。

次回、そのシミュレーションをおこなってみましょう。